# Error Analysis Table

This notebook includes:
1. read results from `result/*`
2. summarize *C1 error analysis*
3. summarize *C2 error analysis*

TODO

In [11]:
from pathlib import Path

c1_res_dir = Path("/home/u5u/kdeng.u5u/spatial-reasoning-of-LMs/result/error-analysis-c1")
c2_res_dir = Path("/home/u5u/kdeng.u5u/spatial-reasoning-of-LMs/result/error-analysis-c2")

In [12]:
import pandas as pd
import jsonlines

def _read_c1_res(res_dir: Path):
    res = []
    for model_dir in res_dir.iterdir():
        if not model_dir.is_dir():
            continue
        res_file = model_dir / "inference.jsonl"
        with jsonlines.open(res_file) as reader:
            for row in reader:
                res.append(row)
                
    return pd.DataFrame(res)

In [28]:
c1_df = _read_c1_res(c1_res_dir)
c1_df["Task"] = "C1"
c2_df = _read_c1_res(c2_res_dir)
c2_df["Task"] = "C2"

In [29]:
a_big_table = pd.concat([c1_df, c2_df], ignore_index=True)

Try to combinate 2 tables

In [30]:
a_big_table.replace({"gpt-4o": "GPT-4o"}, inplace=True)
a_big_table["vlm_id"] = a_big_table["vlm_id"].apply(lambda id: id.split("/")[-1])
vlm_order = [
    "SpaceQwen2.5-VL-3B-Instruct",
    "Idefics3-8B-Llama3",
    "llava-onevision-qwen2-7b-ov-hf",
    "Qwen2.5-VL-7B-Instruct", "Qwen2.5-VL-72B-Instruct", 
    "GPT-4o",
]

a_big_table["vlm_id"] = pd.Categorical(a_big_table["vlm_id"], categories=vlm_order, ordered=True)

a_big_table["prompt_mode"] = a_big_table["prompt_mode"].map({
    0: "zero-shot",
    1: "hint level-1",
    2: "hint level-2",
    3: "hint level-3",
})
prompt_order = ["zero-shot", "hint level-1", "hint level-2", "hint level-3"]
a_big_table["prompt_mode"] = pd.Categorical(a_big_table["prompt_mode"], categories=prompt_order, ordered=True)

a_big_table.rename(columns={"vlm_id": "Model", "prompt_mode": "Strategy"}, inplace=True)

In [31]:
from sklearn.metrics import accuracy_score, f1_score

def acc_f1(group):
    acc = accuracy_score(group["cor_idx"], group["pred"])
    f1 = f1_score(group["cor_idx"], group["pred"], average="weighted")
    return pd.Series({"accuracy": acc, "f1": f1})

In [33]:
nice_table = a_big_table.groupby(["Task", "Model", "Strategy"]).apply(acc_f1).reset_index()
nice_table = nice_table.pivot(index=["Task", "Model"], columns="Strategy", values=["accuracy", "f1"])

/local/user/1483801110/ipykernel_122404/430799841.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  nice_table = a_big_table.groupby(["Task", "Model", "Strategy"]).apply(acc_f1).reset_index()
/local/user/1483801110/ipykernel_122404/430799841.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nice_table = a_big_table.groupby(["Task", "Model", "Strategy"]).apply(acc_f1).reset_index()


In [34]:
nice_table

accuracy                            \
Strategy                            zero-shot hint level-1 hint level-2   
Task Model                                                                
C1   SpaceQwen2.5-VL-3B-Instruct     0.259259     0.518519     0.259259   
     Idefics3-8B-Llama3              0.444444     0.259259     0.222222   
     llava-onevision-qwen2-7b-ov-hf  0.259259     0.555556     0.148148   
     Qwen2.5-VL-7B-Instruct          0.629630     0.629630     0.185185   
     Qwen2.5-VL-72B-Instruct         0.740741     0.692308     0.259259   
     GPT-4o                          0.629630     0.518519     0.444444   
C2   SpaceQwen2.5-VL-3B-Instruct     0.568966     0.568966     0.500000   
     Idefics3-8B-Llama3              0.500000     0.586207     0.551724   
     llava-onevision-qwen2-7b-ov-hf  0.465517     0.517241     0.517241   
     Qwen2.5-VL-7B-Instruct          0.500000     0.517241     0.655172   
     Qwen2.5-VL-72B-Instruct         0.517241     0.568966     0.500000   
     GPT-4o                          0.431034     0.517241     0.586207   

                                                        f1               \
Strategy                            hint level-3 zero-shot hint level-1   
Task Model                                                                
C1   SpaceQwen2.5-VL-3B-Instruct        0.185185  0.227053     0.527816   
     Idefics3-8B-Llama3                 0.148148  0.444444     0.252612   
     llava-onevision-qwen2-7b-ov-hf     0.481481  0.351648     0.571723   
     Qwen2.5-VL-7B-Instruct             0.592593  0.670996     0.624561   
     Qwen2.5-VL-72B-Instruct            1.000000  0.743860     0.732441   
     GPT-4o                             1.000000  0.624542     0.518519   
C2   SpaceQwen2.5-VL-3B-Instruct        0.500000  0.560658     0.564415   
     Idefics3-8B-Llama3                 0.568966  0.494759     0.586207   
     llava-onevision-qwen2-7b-ov-hf     0.603448  0.413117     0.510279   
     Qwen2.5-VL-7B-Instruct             0.586207  0.477953     0.492054   
     Qwen2.5-VL-72B-Instruct            0.413793  0.516092     0.567029   
     GPT-4o                             0.413793  0.405569     0.471506   

                                                               
Strategy                            hint level-2 hint level-3  
Task Model                                                     
C1   SpaceQwen2.5-VL-3B-Instruct        0.249084     0.147510  
     Idefics3-8B-Llama3                 0.237241     0.122299  
     llava-onevision-qwen2-7b-ov-hf     0.174815     0.496212  
     Qwen2.5-VL-7B-Instruct             0.205556     0.592593  
     Qwen2.5-VL-72B-Instruct            0.289811     1.000000  
     GPT-4o                             0.455172     1.000000  
C2   SpaceQwen2.5-VL-3B-Instruct        0.499197     0.477621  
     Idefics3-8B-Llama3                 0.550653     0.458549  
     llava-onevision-qwen2-7b-ov-hf     0.516092     0.535888  
     Qwen2.5-VL-7B-Instruct             0.655172     0.576065  
     Qwen2.5-VL-72B-Instruct            0.497754     0.413793  
     GPT-4o                             0.576065     0.382706

## C1

This notebook:
1. overall summary
2. acc divided by dof

In [ ]:
# rename
# c1_df.rename(columns={"vlm_id": "model"}, inplace=True)
c1_df.replace({"gpt-4o": "GPT-4o"}, inplace=True)
c1_df["vlm_id"] = c1_df["vlm_id"].apply(lambda id: id.split["/"][-1])
vlm_order = [
    "remyxai/SpaceQwen2.5-VL-3B-Instruct",
    "Idefics3-8B-Llama3",
    "llava-onevision-qwen2-7b-ov-hf",
    "Qwen/Qwen2.5-VL-7B-Instruct", "Qwen/Qwen2.5-VL-72B-Instruct", 
    "GPT-4o",
]
c1_df["vlm_id"] = pd.Categorical(c1_df["vlm_id"], categories=vlm_order, ordered=True)

c1_df["prompt_mode"]= c1_df["prompt_mode"].map({
    0: "zero-shot",
    1: "hint level-1",
    2: "hint level-2",
    3: "hint level-3",
})
prompt_order = ["zero-shot", "hint level-1", "hint level-2", "hint level-3"]
c1_df["prompt_mode"] = pd.Categorical(c1_df["prompt_mode"], categories=prompt_order, ordered=True)

c1_df.rename(columns={"vlm_id": "Model", "prompt_mode": "Strategy"}, inplace=True)

In [5]:
summary_table = c1_df.groupby(["Model", "Strategy"]).agg({"is_correct": "mean"})
summary_table.rename(columns={"is_correct": "Accuracy"}, inplace=True)
summary_table.reset_index(inplace=True)
summary_table = summary_table.pivot(index="Model", columns="Strategy", values="Accuracy")
summary_table

/local/user/1483801110/ipykernel_51459/2349648355.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary_table = c1_df.groupby(["Model", "Strategy"]).agg({"is_correct": "mean"})


Strategy,zero-shot,hint level-1,hint level-2,hint level-3
Model,,,,
HuggingFaceM4/Idefics3-8B-Llama3,0.444444,0.259259,0.222222,0.148148
llava-hf/llava-onevision-qwen2-7b-ov-hf,0.259259,0.555556,0.148148,0.481481
remyxai/SpaceQwen2.5-VL-3B-Instruct,0.259259,0.518519,0.259259,0.185185
Qwen/Qwen2.5-VL-7B-Instruct,0.629630,0.629630,0.185185,0.592593
Qwen/Qwen2.5-VL-72B-Instruct,0.740741,0.692308,0.259259,1.000000
meta-llama/Llama-4-Scout-17B-16E-Instruct,0.629630,0.629630,0.555556,1.000000
GPT-4o,0.629630,0.518519,0.444444,1.000000


In [6]:
print(summary_table.to_latex(
    float_format="%.2f",
    column_format="lcccc",
    label="tab:error-aly-3-c1",
    caption="xxx",
))

\begin{table}
\caption{xxx}
\label{tab:error-aly-3-c1}
\begin{tabular}{lcccc}
\toprule
Strategy & zero-shot & hint level-1 & hint level-2 & hint level-3 \\
Model &  &  &  &  \\
\midrule
HuggingFaceM4/Idefics3-8B-Llama3 & 0.44 & 0.26 & 0.22 & 0.15 \\
llava-hf/llava-onevision-qwen2-7b-ov-hf & 0.26 & 0.56 & 0.15 & 0.48 \\
remyxai/SpaceQwen2.5-VL-3B-Instruct & 0.26 & 0.52 & 0.26 & 0.19 \\
Qwen/Qwen2.5-VL-7B-Instruct & 0.63 & 0.63 & 0.19 & 0.59 \\
Qwen/Qwen2.5-VL-72B-Instruct & 0.74 & 0.69 & 0.26 & 1.00 \\
meta-llama/Llama-4-Scout-17B-16E-Instruct & 0.63 & 0.63 & 0.56 & 1.00 \\
GPT-4o & 0.63 & 0.52 & 0.44 & 1.00 \\
\bottomrule
\end{tabular}
\end{table}



## C2
1. summary of error analysis c2

In [7]:
c2_df.replace({"gpt-4o": "GPT-4o"}, inplace=True)
c2_df["vlm_id"] = pd.Categorical(c2_df["vlm_id"], categories=vlm_order, ordered=True)

c2_df["prompt_mode"] = c2_df["prompt_mode"].map({
    0: "zero-shot",
    1: "hint level-1",
    2: "hint level-2",
    3: "hint level-3",
})
prompt_order = ["zero-shot", "hint level-1", "hint level-2", "hint level-3"]
c2_df["prompt_mode"] = pd.Categorical(c2_df["prompt_mode"], categories=prompt_order, ordered=True)

c2_df.rename(columns={"vlm_id": "Model", "prompt_mode": "Strategy"}, inplace=True)

In [8]:
summary_table_c2 = c2_df.groupby(["Model", "Strategy"]).agg({"is_correct": "mean"})
summary_table_c2.rename(columns={"is_correct": "Accuracy"}, inplace=True)
summary_table_c2.reset_index(inplace=True)
summary_table_c2 = summary_table_c2.pivot(index="Model", columns="Strategy", values="Accuracy")
summary_table_c2

/local/user/1483801110/ipykernel_51459/801918176.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary_table_c2 = c2_df.groupby(["Model", "Strategy"]).agg({"is_correct": "mean"})


Strategy,zero-shot,hint level-1,hint level-2,hint level-3
Model,,,,
HuggingFaceM4/Idefics3-8B-Llama3,0.500000,0.586207,0.551724,0.568966
llava-hf/llava-onevision-qwen2-7b-ov-hf,0.465517,0.517241,0.517241,0.603448
remyxai/SpaceQwen2.5-VL-3B-Instruct,0.568966,0.568966,0.500000,0.500000
Qwen/Qwen2.5-VL-7B-Instruct,0.500000,0.517241,0.655172,0.586207
Qwen/Qwen2.5-VL-72B-Instruct,0.517241,0.568966,0.500000,0.413793
meta-llama/Llama-4-Scout-17B-16E-Instruct,0.500000,0.517241,0.517241,0.517241
GPT-4o,0.431034,0.517241,0.586207,0.413793


In [9]:
print(summary_table_c2.to_latex(
    float_format="%.2f",
    column_format="lcccc",
    label="tab:error-aly-3-c2",
    caption="xxx",
))

\begin{table}
\caption{xxx}
\label{tab:error-aly-3-c2}
\begin{tabular}{lcccc}
\toprule
Strategy & zero-shot & hint level-1 & hint level-2 & hint level-3 \\
Model &  &  &  &  \\
\midrule
HuggingFaceM4/Idefics3-8B-Llama3 & 0.50 & 0.59 & 0.55 & 0.57 \\
llava-hf/llava-onevision-qwen2-7b-ov-hf & 0.47 & 0.52 & 0.52 & 0.60 \\
remyxai/SpaceQwen2.5-VL-3B-Instruct & 0.57 & 0.57 & 0.50 & 0.50 \\
Qwen/Qwen2.5-VL-7B-Instruct & 0.50 & 0.52 & 0.66 & 0.59 \\
Qwen/Qwen2.5-VL-72B-Instruct & 0.52 & 0.57 & 0.50 & 0.41 \\
meta-llama/Llama-4-Scout-17B-16E-Instruct & 0.50 & 0.52 & 0.52 & 0.52 \\
GPT-4o & 0.43 & 0.52 & 0.59 & 0.41 \\
\bottomrule
\end{tabular}
\end{table}

